In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib

import ribasim

import ribasim_lumping
# from ribasim_lumping.ribasim_model_results import plot_results_basin_ribasim_model, \
#     plot_results_basins_ribasim_model, get_ribasim_basin_data_from_model, read_ribasim_model_results

import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow as pa
import datetime

import warnings

warnings.simplefilter("ignore")
pd.options.mode.chained_assignment = None

In [ ]:
%load_ext autoreload
%autoreload 2

##### SET-UP NETWORK

In [ ]:
# define network name, base dir
network_name            = "tki_zutphen_berkel"
base_dir                = Path("..\\..\\..\\ribasim_lumping_data\\")
# base_dir                = Path("..\\..\\..\\..\\..\\..\\ribasim_lumping_data\\")
# directory D-HYDRO basis calculations
dhydro_basis_dir        = Path(base_dir, network_name, "2_dhydro\\") # zelfde directory, want gebruik zelfde som
# directory D-HYDRO results calculations
dhydro_results_dir      = Path(base_dir, network_name, "2_dhydro\\") # zelfde directory, want gebruik zelfde som
# directory results
results_dir             = Path(base_dir, network_name, "3_ribasim\\")
# batch file used to calculate volume D-Hydro module per GridPoint
dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
dhydro_volume_tool_increment = 0.1
# link to ribasim_executable
path_ribasim_executable = "..\\..\\..\\ribasim_cli_windows\\bin\\ribasim.exe"

Start ribasim network

In [ ]:
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    path_ribasim_executable=path_ribasim_executable,
    crs=28992
)
# network.export_to_geopackage()

Load areas (discharge units: afwaterende eenheden)

In [ ]:
areas_dir               = Path(base_dir, network_name, "1_data", "areas")
areas_file_name         = f"{network_name}_afw_eenheden.shp"
areas_id_column         = "GFEIDENT"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)
network.plot();

Read network data and extract objects and settings:
- extract network and weirs/pumps/culverts/etc
- read structure settings
- read laterals/boundaries/etc from basis simulations

In [ ]:
network.add_basis_network(
    source_type='dhydro',
    set_name="winter",
    set_start_month=10,
    set_start_day=1,
    model_dir=Path(dhydro_basis_dir, "afvoergolf"),
    simulation_name="tki_zuthpen_berkel_afvoergolf.dsproj_data",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=True,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
network.add_basis_network(
    source_type='dhydro',
    set_name="zomer",
    set_start_month=10,
    set_start_day=1,
    model_dir=Path(dhydro_basis_dir, "afvoergolf"),
    simulation_name="tki_zuthpen_berkel_afvoergolf.dsproj_data",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=True,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
network.plot();

In [ ]:
network.pumps_gdf.head()

In [ ]:
network.weirs_gdf.iloc[0]

In [ ]:
network.culverts_gdf.iloc[0]

In [ ]:
network.orifices_gdf.iloc[0]

In [ ]:
network.uniweirs_gdf.iloc[0]

In [ ]:
structure_specs_dir_path = Path(base_dir, network_name, "1_data", "control")
network.export_or_update_all_ribasim_structures_specs(
    structure_specs_dir_path=structure_specs_dir_path
);

Select simulation sets and extract all simulation results using xugrid/ugrid

In [ ]:
network.add_simulation_set(
    source_type='dhydro',
    set_name="winter",
    model_dir=Path(dhydro_results_dir, "winter"),
    simulation_names=["tki_zuthpen_berkel_basis_winter.dsproj_data"],
    simulation_ts=pd.date_range("2000-01-06 23:00", periods=9, freq="6D"),
);
network.add_simulation_set(
    source_type='dhydro',
    set_name="zomer",
    model_dir=Path(dhydro_results_dir, "zomer"),
    simulation_names=["tki_zuthpen_berkel_basis_zomer.dsproj_data"],
    simulation_ts=pd.date_range("2000-01-06 23:00", periods=9, freq="6D"),
);

#### SET-UP SIMULATION
simulation code and temporal settings

In [ ]:
simulation_code = "tki_zutphen_berkel"
# Definition settings changes over time
settings_time_variation = False
# in case of no variation over time. set set_name ("winter", "zomer", ...)
settings_set_name = 'winter'

Defition Lateral inflow over time
- method_laterals = 1: laterals basis network
- method_laterals = 2: laterals per discharge area > required: run function read_areas_laterals_timeseries(path_to_excel)
- method_laterals = 3: homogeneous discharge > required: define laterals_drainage_per_ha (pandas series) in liters/second/hectare

In [ ]:
network.method_laterals = 2

# in case of 2: give direction to csv file with timeseries per area -> column names according to areas[areas_id_column]
areas_laterals_path = Path(base_dir, network_name, "1_data", "laterals", "laterals_areas.csv")
network.read_areas_laterals_timeseries(areas_laterals_path, sep=";", dayfirst=True)

fig, ax = plt.subplots(1,1)
network.laterals_areas_data.sum(axis=1).plot(ax=ax)
ax.hlines(0, xmin=network.laterals_areas_data.index[0], xmax=network.laterals_areas_data.index[-1])

Definition boundaries
- method_boundaries = 1: use boundaries basis network, 
- method_boundaries = 2: use timeseries from imported csv, 
- method_boundaries = 3: use fixed boundaries

In [ ]:
network.method_boundaries = 2

boundaries_timeseries_path = Path(base_dir, network_name, "1_data", "boundaries", "boundary_timeseries.csv")
network.read_boundaries_timeseries_data(boundaries_timeseries_path, skiprows=1, sep=";")
# small adjustment to datetime
network.boundaries_timeseries_data.index = network.boundaries_timeseries_data.index.floor(freq="D")
network.boundaries_timeseries_data.plot();

Changes to timeseries data:
- start and end time equal to laterals
- backward fill of data from Almen (1389)

In [ ]:
network.boundaries_timeseries_data = network.boundaries_timeseries_data.loc[
    network.laterals_areas_data.index[0]:network.laterals_areas_data.index[-1]
].bfill()
network.boundaries_timeseries_data.plot();

Definition initial water levels (refer to simulation name and index timestep)
- method_initial_waterlevels = 1: based on one of the basis simulations
- method_initial_waterlevels = 2: based on one of the simulations provided
- method_initial_waterlevels = 3: based on areas, define a column (zomerpeil/winterpeil) to be used (NOT YET IMPLEMENTED)

In [ ]:
network.method_initial_waterlevels = 2

# give name of simulation plus the selected timestep (int)
network.initial_waterlevels_set_name = "tki_zuthpen_berkel_afvoergolf.dsproj_data"
network.initial_waterlevels_timestep = 3

#### ADD SPLIT NODES
use add_split_nodes function to add splitnodes. 
Select all weirs/pumps/uniweirs/orifices and/or select individual structures/objects to be included or excluded.

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    orifices=True,
    culverts=False,
    edges=False,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kdu_DR80770020', # duiker afvoergebiedje noordzijde Berkel
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
        'kdu_DR80740070', # inlaat twentekanaal
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', 'BBypass_Besselink_6', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
        # 'kst_ST80830045', # weir tussen uitstroompunten Zutphen
        # 'kst_ST80820031', 'kst_ST80820042', # test
    ], 
    edge_ids_to_include=[1801],
);

Conversion of choosen split_nodes to ribasim-type. Select standard conversion for objects and individual exceptions

In [ ]:
split_node_type_conversion = dict(
    weir="TabulatedRatingCurve",
    universalWeir="TabulatedRatingCurve",
    pump="Pump",
    culvert="TabulatedRatingCurve",
    edge="ManningResistance",
    orifice="Outlet",
    boundary_connection="ManningResistance"
)

# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = dict(
    kdu_DR80740070='ManningResistance',
    # duikers voor wijk Zuidwijken
    kdu_DR80940046='ManningResistance', 
    kdu_DR80950043='ManningResistance', 
    kdu_DR80950151='ManningResistance', 
)

##### NETWORK GENERATION

In [ ]:
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);

In [ ]:
ribasim_model = network.generate_ribasim_model_complete(
    set_name="winter",
    dummy_model=True,
    saveat=3600,
)

In [ ]:
# network.plot_tabulated_rating_curves()

In [ ]:
network.ribasim_model.tabulated_rating_curve.static.df

In [ ]:
network.ribasim_model.basin.state

In [ ]:
network.ribasim_model.tabulated_rating_curve.static.df